### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
clf = DecisionTreeClassifier()

In [128]:
a = np.array([1,2,3,4,5,3,3,3,4,4,25])
b = np.array([1,2,3,4,5,3,3,3,4,4,5,1,1,21,1])
a = np.array(a)
b = np.array(b)

np.concatenate((a, b), axis=0)

array([ 1,  2,  3,  4,  5,  3,  3,  3,  4,  4, 25,  1,  2,  3,  4,  5,  3,
        3,  3,  4,  4,  5,  1,  1, 21,  1])

In [4]:
feature_ids = [i for i in range(10)]
np.random.shuffle(feature_ids)

In [653]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
    
    
    def __func_gini(self, y_in_v):
        N = y_in_v.shape[0]
        p = np.unique(y_in_v, return_counts=True)[1] / N
        return 1 - (p ** 2).sum()
        
        
    def __gini(self, y_left, y_right):
        
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r
        return (self.__func_gini(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_gini(y_left)
                - N_r / N_m * self.__func_gini(y_right))
    
    
    
    def __func_entropy(self, y_in_v):
        
        N = y_in_v.shape[0]
        
        p = np.unique(y_in_v, return_counts=True)[1] / N
        return -(p * np.log2(p)).sum()
        
    def __entropy(self, y_left, y_right):
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r

        return (self.__func_entropy(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_entropy(y_left)
                - N_r / N_m * self.__func_entropy(y_right))

    
    
    
    def __func_misclass(self, y_in_v):
        classes, classes_cnt = np.unique(y_in_v, return_counts=True)
        k = classes[classes_cnt.argmax()]
        return 1. / y_in_v.shape[0] * (y_in_v == k).sum()
    
    def __misclass(self, y_left, y_right):
        
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r
        return (self.__func_misclass(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_misclass(y_left)
                - N_r / N_m * self.__func_misclass(y_right))

    
    
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        thresholds = x

        gains = np.array([self.G_function(y[x <= thr], y[x > thr]) for thr in thresholds])
        if gains.min() == 10:
            return np.inf, None
        ind = gains.argmin()
        return gains[ind], (thresholds[ind] + thresholds[ind-1]) / 2. 

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        
        if ((self.max_depth is not None and self.max_depth < depth)
                or (y.shape[0] < self.min_samples_split)):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        feature_ids = self.get_feature_ids(x.shape[1])
        threshold = np.array([self.__find_threshold(x[:, feature_id], y)
                             for feature_id in feature_ids])
        
        best_split_id = np.argmin(threshold[:, 0])
        gs, best_split = threshold[best_split_id]

        if best_split is None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        x_l, x_r, y_l, y_r = \
            self.__div_samples(x, y, feature_ids[best_split_id], best_split)
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_ids[best_split_id],
                              best_split)

        

        self.__fit_node(x_l, y_l, 2*node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2*node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [654]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion="gini")
clf = DecisionTreeClassifier(min_samples_split=2)

In [655]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [656]:
%time clf.fit(X_train, y_train)

CPU times: user 1.56 ms, sys: 41 µs, total: 1.6 ms
Wall time: 928 µs


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [657]:
%time my_clf.fit(X_train, y_train)

IndexError: index 6 is out of bounds for axis 0 with size 6

## Проверка качества работы на wine

In [650]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [652]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.6484848484848484

## Подготовка данных Speed Dating Data 

## Проверка скорости работы на Speed Dating Data 

In [ ]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

## Проверка качества работы на Speed Dating Data

In [ ]:
# тут должен быть код типа %time clf.fit(X_train, y_train)

In [ ]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)

## Задание 3

## Задание 4

## Задание 5

In [ ]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
    
    
    def __func_gini(self, y_in_v):
        N = y_in_v.shape[0]
        p = np.unique(y_in_v, return_counts=True)[1] / N
        return 1 - (p ** 2).sum()
        
        
    def __gini(self, y_left, y_right):
        
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r
        return (self.__func_gini(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_gini(y_left)
                - N_r / N_m * self.__func_gini(y_right))
    
    
    
    def __func_entropy(self, y_in_v):
        
        N = y_in_v.shape[0]
        
        p = np.unique(y_in_v, return_counts=True)[1] / N
        return -(p * np.log2(p)).sum()
        
    def __entropy(self, y_left, y_right):
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r

        return (self.__func_entropy(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_entropy(y_left)
                - N_r / N_m * self.__func_entropy(y_right))

    
    
    
    def __func_misclass(self, y_in_v):
        classes, classes_cnt = np.unique(y_in_v, return_counts=True)
        k = classes[classes_cnt.argmax()]
        return 1. / y_in_v.shape[0] * (y_in_v == k).sum()
    
    def __misclass(self, y_left, y_right):
        
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r
        return (self.__func_misclass(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_misclass(y_left)
                - N_r / N_m * self.__func_misclass(y_right))

    
    
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        thresholds = x

        gains = np.array([self.G_function(y[x <= thr], y[x > thr]) for thr in thresholds])
        if gains.min() == 10:
            return None
        #print(thresholds[gains.argmin()])
        print(thresholds)
        return thresholds[gains.argmin()]
        

    def __fit_node(self, x, y, node_id, depth):
        
        if ((y.shape[0] < self.min_samples_split) or
            (self.max_depth is not None and self.max_depth < depth)):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return
        
        x = np.array(x)
        y = np.array(y)
        main_features = self.get_feature_ids(x.shape[1])
        
        
        thresholds = [self.__find_threshold(x[:,num_feature], y) 
                               for num_feature in main_features]
        
        
        gains = []
        for i in range(len(main_features)):
            if thresholds[i] is not None:
                gains.append(self.G_function(y[x[:,main_features[i]] <= thresholds[i]], 
                                 y[x[:,main_features[i]] > thresholds[i]]))
            else:
                gains.append(10)
        
        gains = np.array(gains)
        
        if gains.min() == 10:
            
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return
        
        index = np.argmin(gains)
        
        left_indexes = (x[:,index] <= thresholds[index])
        right_indexes = (x[:,index] > thresholds[index])
        
        
        self.tree[node_id] = (self.NON_LEAF_TYPE, main_features[index],
                              thresholds[index])
        print(y[left_indexes].shape, y[right_indexes].shape)
        self.__fit_node(x[left_indexes, :], y[left_indexes], 2 * node_id + 1, depth + 1)
        self.__fit_node(x[right_indexes, :], y[right_indexes], 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)